# Step by step to do Super Resolution Convolutional Neural Network
## Implement an image super-resolution technique in TensorFlow

> I'm still not done on doing explanatory journey with this code. Some still taken from the original website. 

This step-by-step approach will be run through the methodology based on this link below

https://dzlab.github.io/notebooks/tensorflow/generative/artistic/2021/05/10/Super_Resolution_SRCNN.html

The run through methodology I used made by : [dzlab](https://github.com/dzlab/notebooks/blob/master/_notebooks/2021-05-10-Super_Resolution_SRCNN.ipynb) 

In [1]:
# Just make sure this REPL run thesis_super_resolution

import os
print(os.environ['CONDA_DEFAULT_ENV'])

thesis_super_resolution_p3_10


In [2]:
# Importing Packages
import os
import pathlib
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import *

In [ ]:
# Random seed for reproducibility???
SEED = 31
np.random.seed(SEED)

In [ ]:
# Constants

# Dataset-related constant

file_path = (pathlib.Path('/content') / 'images' / '*.png')
file_pattern = str(file_path)
dataset_path = [*glob(file_pattern)]

SUBSET_SIZE = 1000
dataset_path = np.random.choice(dataset_path, SUBSET_SIZE)

# Parameter-based constant
SCALE = 2.0
INPUT_DIM = 33
LABEL_SIZE = 21
PAD = int((INPUT_DIM - LABEL_SIZE) / 2.0)
STRIDE = 14

In [ ]:
# Example Immage from the dataset

path = np.random.choice(dataset_path)
img = plt.imread(path)
plt.imshow(img)

## Support Functions 
In the references, I saw some support functions to help with data pre-processing:

### 1. Resize image
As its name, this function will resize an image.

This function requires these parameters below:

| Parameter | Description |
|---|---|
| `image_array` | This parameter specified for a converted image array. So basically, the image that already transformed into multi-dimensional array. |
| `factor` | The percentage of resized image. Giving value of below than 1 means that the resized image will be bigger than the original image. |

This function return a resized image array data.

### 2. Downsize Upsize Image
This function will generate low resolution image by downsize and upsizing it.

In [ ]:
# resize_image
def resize_image(image_array, factor):
    original_image = Image.fromarray(image_array)

    new_size = np.array(original_image.size) * factor
    new_size = new_size.astype(np.int32)
    new_size = tuple(new_size)

    resized = original_image.resize(new_size)
    resized = img_to_array(resized)
    resized = resized.astype(np.uint8)
    
    return resized

def downsize_upsize_image(image, scale):
    scaled = resize_image(image, 1.0 / scale)
    scaled = resize_image(scaled, scale) # In the reference, the scale is divided by 1.0. What changes over it?

    return scaled

def tight_crop_image(image, scale):
    height, width = image.shape[:2]

    width -= int(width % scale)
    height -= int(height % scale)

    return image[:height, :width]

def crop_input(image, x, y)
    x_slice = slice(x, x + INPUT_DIM)
    y_slice = slice(y, y + INPUT_DIM)
    return image[y_slice, x_slice]

def crop_output(image, x, y)
    x_slice = slice(x + PAD, x + PAD + LABEL_SIZE)
    y_slice = slice(y + PAD, y + PAD + LABEL_SIZE)
    
    return image[y_slice, x_slice]

## Getting data from the disk

-> I'm still not sure what to do with this step, but it says:

> Now, lets build the dataset by reading the input images, generating a low resolution version, sliding a window on this low resolution image as well as the original image to generate patches for training. We will save the patches to disk and later build a training data generator that will load them from > disk in batches.

In [ ]:
for image_path in tqdm(dataset_path):
    filename = pathlib.Path(image_path).stem
    image = load_img(image_path)
    image = img_to_array(image)
    image = image.astype(np.uint8)
    image = tight_crop_image(image, SCALE)
    scaled = downsize_upsize_image(image, SCALE)

    height, width = image.shape[:2]

    for y in range(0, height - INPUT_DIM + 1, STRIDE):
        for x in range(0, width - INPUT_DIM + 1, STRIDE):
            crop = crop_input(scaled, x, y)
            target = crop_output(image, x, y)
            np.save(f'data/{filename}_{x}_{y}_input.np', crop)
            np.save(f'data/{filename}_{x}_{y}_output.np', target)



> We cannot hold all the patches in memory hence we saved to disk in the previous step. Now we need a dataset loader that will load a patch and its label and feed them to the network during traning in batches. This is achieved with the PatchesDataset class (check this example to learn more about generators - [link](https://dzlab.github.io/dltips/en/keras/data-generator/)).

In [ ]:
class PatchesDataset(tf.keras.utils.Sequence):
    def __init__(self, batch_size, *args, **kwargs):
        self.batch_size = batch_size
        self.input = [*glob('data/*_input.np.npy')]
        self.output = [*glob('data/*_output.np.npy')]
        self.input.sort()
        self.output.sort()
        self.total_data = len(self.input)

    def __len__(self):
        # returns the number of batches
        return int(self.total_data / self.batch_size)

    def __getitem__(self, index):
        # returns one batch
        indices = self.random_indices()
        input = np.array([np.load(self.input[idx]) for idx in indices])
        output = np.array([np.load(self.output[idx]) for idx in indices])
        return input, output

    def random_indices(self):
        return np.random.choice(list(range(self.total_data)), self.batch_size, p=np.ones(self.total_data)/self.total_data)

> Define a batch size based on how much memory available on your GPU and create an instance of the dataset generator.

In [ ]:
BATCH_SIZE = 1024
train_ds = PatchesDataset(BATCH_SIZE)
len(train_ds)

> You can see the shape of the training batches

In [ ]:
input, output = train_ds[0]
input.shape, output.shape

## Model 

> The architecture of the SRCNN model is very simple, it has only convolutional layers, one to downsize the input and extract image features and a later one to upside to generate the output image. The following helper function is used to create an instance of the model.

In [ ]:
def create_model(height, width, depth):
    input = Input(shape=(height, width, depth))
    x = Conv2D(filters=64, kernel_size=(9, 9), kernel_initializer='he_normal')(input)
    x = ReLU()(x)
    x = Conv2D(filters=32, kernel_size=(1, 1), kernel_initializer='he_normal')(x)
    x = ReLU()(x)
    output = Conv2D(filters=depth, kernel_size=(5, 5), kernel_initializer='he_normal')(x)
    return Model(input, output)

> To train the network we will use Adam as optimizer with learning rate decay. Also, as the problem we try to train the network for is a regression problem (we want predict the high resolution pixels) we pick MSE as a loss function, this will make the model learn the filters that correctly map patches from low to high resolution.

In [ ]:
EPOCHS = 12
optimizer = Adam(learning_rate=1e-3, decay=1e-3 / EPOCHS)
model = create_model(INPUT_DIM, INPUT_DIM, 3)
model.compile(loss='mse', optimizer=optimizer)

> You can see how the model is small but astonishly it will be able to achieve great results once trained for enough time, we will train it for 12 epochs

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, show_shapes = True, rankdir='LR')

> Create a callback that saves the model's weights

In [ ]:
checkpoint_path = "training/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

Now finally, we can train the network

In [ ]:
model.fit(train_ds, epochs=EPOCHS, callbacks=[cp_callback])